In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window


In [ ]:
spark = (
      SparkSession
      .builder
      .master('local[*]') #Usando todos os cores da maquina
      .appName('spark_1') #Dando nome a sessão
      .getOrCreate()
)

In [ ]:
df = (
    spark
    .read
    .option('header', 'true') #Possui cabeçalho (nome da coluna)
    .option('delimiter', ';') #O delimitador do nosso csv é ;
    .option('inferSchema', 'true') #O spark já inferir os tipos dos dados das colunas
    .option('enconding', 'ISO-8859-1') #ISO de arquivo csv
    .format('csv') #Informando que é um arquivo csv
    .csv('/content/drive/MyDrive/ESTUDOS/pyspark/precos-gasolina-etanol-08.csv') #Informando o arquivo
)

exibindo o df (df.show)

In [ ]:
df.show(30)

+--------------+--------------+---------+--------------------+-------------------+--------------------+----------+--------------+-----------------+---------+------------------+--------------+--------------+---------------+-----------------+--------+
|Regiao - Sigla|Estado - Sigla|Municipio|             Revenda|    CNPJ da Revenda|         Nome da Rua|Numero Rua|   Complemento|           Bairro|      Cep|           Produto|Data da Coleta|Valor de Venda|Valor de Compra|Unidade de Medida|Bandeira|
+--------------+--------------+---------+--------------------+-------------------+--------------------+----------+--------------+-----------------+---------+------------------+--------------+--------------+---------------+-----------------+--------+
|             N|            AM|PARINTINS|F.J. COMERCIO DE ...| 09.368.602/0001-70|         RUA PARAIBA|        41|CONJ  MACURANY|   RAIMUNDO MUNIZ|69151-456|          GASOLINA|    01/08/2024|          7,68|           NULL|       R$ / litro|  BRANCA|


verificar tipos dos dados das colunas (seria o df.show())

In [ ]:
df.printSchema()

root
 |-- Regiao - Sigla: string (nullable = true)
 |-- Estado - Sigla: string (nullable = true)
 |-- Municipio: string (nullable = true)
 |-- Revenda: string (nullable = true)
 |-- CNPJ da Revenda: string (nullable = true)
 |-- Nome da Rua: string (nullable = true)
 |-- Numero Rua: string (nullable = true)
 |-- Complemento: string (nullable = true)
 |-- Bairro: string (nullable = true)
 |-- Cep: string (nullable = true)
 |-- Produto: string (nullable = true)
 |-- Data da Coleta: string (nullable = true)
 |-- Valor de Venda: string (nullable = true)
 |-- Valor de Compra: string (nullable = true)
 |-- Unidade de Medida: string (nullable = true)
 |-- Bandeira: string (nullable = true)



verificando valores nulos

In [ ]:
#como o df é pequeno, podemos converter para pandas e executar comandos pandas, pois o pyspark não contém um verificador direto de nulos
df.toPandas().isna().sum()

,0
Regiao - Sigla,0
Estado - Sigla,0
Municipio,0
Revenda,0
CNPJ da Revenda,0
Nome da Rua,0
Numero Rua,12
Complemento,35292
Bairro,56
Cep,0


In [ ]:
#agora, fazendo um loop for para contarmos quandos nulos há em cada coluna com pyspark
#estamos usando o for pois não há funcão nativa para contar nulos como no pandas

for coluna in df.columns:
  print(coluna, df.filter(df[coluna].isNull()).count())

Regiao - Sigla 0
Estado - Sigla 0
Municipio 0
Revenda 0
CNPJ da Revenda 0
Nome da Rua 0
Numero Rua 12
Complemento 35292
Bairro 56
Cep 0
Produto 0
Data da Coleta 0
Valor de Venda 0
Valor de Compra 45119
Unidade de Medida 0
Bandeira 0


renomeando colunas

In [ ]:
df_2 = df.withColumnRenamed('Regiao - Sigla', 'regiao')\
      .withColumnRenamed('Estado - Sigla', 'estado')\
      .withColumnRenamed('Municipio', 'municipio')\
      .withColumnRenamed('Revenda', 'revenda')\
      .withColumnRenamed('CNPJ da Revenda', 'cnpj')\
      .withColumnRenamed('Nome da Rua', 'nomerua')\
      .withColumnRenamed('Numero da Rua', 'numerorua')\
      .withColumnRenamed('Complemento', 'complemento')\
      .withColumnRenamed('Bairro', 'bairro')\
      .withColumnRenamed('Cep', 'cep')\
      .withColumnRenamed('Produto', 'produto')\
      .withColumnRenamed('Data da Coleta', 'datacoleta')\
      .withColumnRenamed('Valor de Venda', 'vlrvenda')\
      .withColumnRenamed('Valor de Compra', 'vlrcompra')\
      .withColumnRenamed('Unidade de Medida', 'unidmedida')\
      .withColumnRenamed('Bandeira', 'bandeira')\

df_2.show(3)

+------+------+---------+--------------------+-------------------+--------------------+----------+--------------+--------------+---------+------------------+----------+--------+---------+----------+--------+
|regiao|estado|municipio|             revenda|               cnpj|             nomerua|Numero Rua|   complemento|        bairro|      cep|           produto|datacoleta|vlrvenda|vlrcompra|unidmedida|bandeira|
+------+------+---------+--------------------+-------------------+--------------------+----------+--------------+--------------+---------+------------------+----------+--------+---------+----------+--------+
|     N|    AM|PARINTINS|F.J. COMERCIO DE ...| 09.368.602/0001-70|         RUA PARAIBA|        41|CONJ  MACURANY|RAIMUNDO MUNIZ|69151-456|          GASOLINA|01/08/2024|    7,68|     NULL|R$ / litro|  BRANCA|
|     N|    AM|PARINTINS|F.J. COMERCIO DE ...| 09.368.602/0001-70|         RUA PARAIBA|        41|CONJ  MACURANY|RAIMUNDO MUNIZ|69151-456|GASOLINA ADITIVADA|01/08/2024|

fazendo um replace

In [ ]:
df_2 = df_2.withColumn('vlrvenda', regexp_replace('vlrvenda', ',', '.'))

alterando o tipo dos dados

In [ ]:
df_3 = df_2\
        .withColumn('datacoleta', to_date('datacoleta', format='dd/MM/yyyy'))\
        .withColumn('vlrvenda', df_2['vlrvenda'].cast(FloatType()))\

df_3.printSchema()

root
 |-- regiao: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- revenda: string (nullable = true)
 |-- cnpj: string (nullable = true)
 |-- nomerua: string (nullable = true)
 |-- Numero Rua: string (nullable = true)
 |-- complemento: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- cep: string (nullable = true)
 |-- produto: string (nullable = true)
 |-- datacoleta: date (nullable = true)
 |-- vlrvenda: float (nullable = true)
 |-- vlrcompra: string (nullable = true)
 |-- unidmedida: string (nullable = true)
 |-- bandeira: string (nullable = true)



selecionando colunas

In [ ]:
#Selecionando apenas uma coluna
df_3.select('regiao').show(5)

+------+
|regiao|
+------+
|     N|
|     N|
|    NE|
|    NE|
|    NE|
+------+
only showing top 5 rows



In [ ]:
#selecionando valores únicos da coluna
df_3.select('regiao').distinct().show(10)

+------+
|regiao|
+------+
|    NE|
|     N|
|     S|
|    SE|
|    CO|
+------+



In [ ]:
#Por que usar 'col' se podemos chamar a coluna apenas com o 'select'
#Pois com o 'col', podemos usar uma função em cima da coluna e apenas com o select não
df_3.select(col('regiao')).distinct().show()

+------+
|regiao|
+------+
|    NE|
|     N|
|     S|
|    SE|
|    CO|
+------+



In [ ]:
#nesse caso, não retorna erro, pois agora além de chamar a coluna, podemos executar funções sobre a colua
df_3.select(col('regiao').alias('novo_nome')).distinct().show()

+---------+
|novo_nome|
+---------+
|       NE|
|        N|
|        S|
|       SE|
|       CO|
+---------+



In [ ]:
#Assim como no SQL, podemos chamar as colunas de acordo com a orderm que queros exibilas e não na ordem do df original
df_3.select(col('bairro'),
            col('complemento'),
            col('regiao'))\
            .show(3)

+--------------+--------------+------+
|        bairro|   complemento|regiao|
+--------------+--------------+------+
|RAIMUNDO MUNIZ|CONJ  MACURANY|     N|
|RAIMUNDO MUNIZ|CONJ  MACURANY|     N|
|         JULHO|          NULL|    NE|
+--------------+--------------+------+
only showing top 3 rows



In [ ]:
#ou, já que queremos só exibir, podemos usar apenas o select
df_3.select('bairro', 'complemento', 'regiao').show(3)

+--------------+--------------+------+
|        bairro|   complemento|regiao|
+--------------+--------------+------+
|RAIMUNDO MUNIZ|CONJ  MACURANY|     N|
|RAIMUNDO MUNIZ|CONJ  MACURANY|     N|
|         JULHO|          NULL|    NE|
+--------------+--------------+------+
only showing top 3 rows



filtrando colunas

In [ ]:
#forma "correta", porém limitada de aplicar um filtro no pyspark
df_3.filter('bairro = "RAIMUNDO MUNIZ"').show()

+------+------+---------+--------------------+-------------------+-----------+----------+--------------+--------------+---------+------------------+----------+--------+---------+----------+--------+
|regiao|estado|municipio|             revenda|               cnpj|    nomerua|Numero Rua|   complemento|        bairro|      cep|           produto|datacoleta|vlrvenda|vlrcompra|unidmedida|bandeira|
+------+------+---------+--------------------+-------------------+-----------+----------+--------------+--------------+---------+------------------+----------+--------+---------+----------+--------+
|     N|    AM|PARINTINS|F.J. COMERCIO DE ...| 09.368.602/0001-70|RUA PARAIBA|        41|CONJ  MACURANY|RAIMUNDO MUNIZ|69151-456|          GASOLINA|2024-08-01|    7.68|     NULL|R$ / litro|  BRANCA|
|     N|    AM|PARINTINS|F.J. COMERCIO DE ...| 09.368.602/0001-70|RUA PARAIBA|        41|CONJ  MACURANY|RAIMUNDO MUNIZ|69151-456|GASOLINA ADITIVADA|2024-08-01|    7.68|     NULL|R$ / litro|  BRANCA|
|    

In [ ]:
df_3.select(col('produto')).distinct().show()

+------------------+
|           produto|
+------------------+
|GASOLINA ADITIVADA|
|            ETANOL|
|          GASOLINA|
+------------------+



In [ ]:
#forma correta de aplicar um filtro em uma coluna
df_3.filter(
    (col('municipio') == "SAO PAULO") &
    ((col('produto') == "GASOLINA") | (col('produto') == "ETANOL")) &
    (col('vlrvenda') > 5)
).show(20)

+------+------+---------+--------------------+-------------------+--------------------+----------+--------------------+--------------------+---------+--------+----------+--------+---------+----------+--------+
|regiao|estado|municipio|             revenda|               cnpj|             nomerua|Numero Rua|         complemento|              bairro|      cep| produto|datacoleta|vlrvenda|vlrcompra|unidmedida|bandeira|
+------+------+---------+--------------------+-------------------+--------------------+----------+--------------------+--------------------+---------+--------+----------+--------+---------+----------+--------+
|    SE|    SP|SAO PAULO|AUTO POSTO CAVALL...| 04.885.965/0001-60|AVENIDA NOSSA SEN...|      1145|                NULL|      FREGUESIA DO Ó|02715-000|GASOLINA|2024-08-05|    5.78|     NULL|R$ / litro|IPIRANGA|
|    SE|    SP|SAO PAULO|GAIVOTA SERVICOS ...| 02.756.935/0001-28|     AVENIDA COTOVIA|       555|                NULL|               MOEMA|04517-001|GASOLINA|2

Criando novas colunas


In [ ]:
#Se eu colocar uma coluna que já existe no .withColumn('coluna_existente), ele vai alterar essa coluna já existente

#Criando uma coluna nova, onde concateno dois valores

#Nesse caso não estou criando, estou apenas "exibindo" o mesmo df, por exemplo, se eu der um df_3.show() em baixo, ele não terá a coluna concatenar,
    #pois não reescrevi o antigo df com a coluna nova, apenas exibi
df_3.withColumn('contat', concat(col('regiao'), col('estado')))

DataFrame[regiao: string, estado: string, municipio: string, revenda: string, cnpj: string, nomerua: string, Numero Rua: string, complemento: string, bairro: string, cep: string, produto: string, datacoleta: date, vlrvenda: float, vlrcompra: string, unidmedida: string, bandeira: string, contat: string]

In [ ]:
df_3.show()

+------+------+---------+--------------------+-------------------+--------------------+----------+--------------+----------------+---------+------------------+----------+--------+---------+----------+--------+
|regiao|estado|municipio|             revenda|               cnpj|             nomerua|Numero Rua|   complemento|          bairro|      cep|           produto|datacoleta|vlrvenda|vlrcompra|unidmedida|bandeira|
+------+------+---------+--------------------+-------------------+--------------------+----------+--------------+----------------+---------+------------------+----------+--------+---------+----------+--------+
|     N|    AM|PARINTINS|F.J. COMERCIO DE ...| 09.368.602/0001-70|         RUA PARAIBA|        41|CONJ  MACURANY|  RAIMUNDO MUNIZ|69151-456|          GASOLINA|2024-08-01|    7.68|     NULL|R$ / litro|  BRANCA|
|     N|    AM|PARINTINS|F.J. COMERCIO DE ...| 09.368.602/0001-70|         RUA PARAIBA|        41|CONJ  MACURANY|  RAIMUNDO MUNIZ|69151-456|GASOLINA ADITIVADA|2

In [ ]:
#Fazendo um concat_ws (with separetor)

df_3.withColumn('concat', concat_ws('_', col('regiao'), col('estado')))

DataFrame[regiao: string, estado: string, municipio: string, revenda: string, cnpj: string, nomerua: string, Numero Rua: string, complemento: string, bairro: string, cep: string, produto: string, datacoleta: date, vlrvenda: float, vlrcompra: string, unidmedida: string, bandeira: string, concat: string]

In [ ]:
df_3.show()

+------+------+---------+--------------------+-------------------+--------------------+----------+--------------+----------------+---------+------------------+----------+--------+---------+----------+--------+
|regiao|estado|municipio|             revenda|               cnpj|             nomerua|Numero Rua|   complemento|          bairro|      cep|           produto|datacoleta|vlrvenda|vlrcompra|unidmedida|bandeira|
+------+------+---------+--------------------+-------------------+--------------------+----------+--------------+----------------+---------+------------------+----------+--------+---------+----------+--------+
|     N|    AM|PARINTINS|F.J. COMERCIO DE ...| 09.368.602/0001-70|         RUA PARAIBA|        41|CONJ  MACURANY|  RAIMUNDO MUNIZ|69151-456|          GASOLINA|2024-08-01|    7.68|     NULL|R$ / litro|  BRANCA|
|     N|    AM|PARINTINS|F.J. COMERCIO DE ...| 09.368.602/0001-70|         RUA PARAIBA|        41|CONJ  MACURANY|  RAIMUNDO MUNIZ|69151-456|GASOLINA ADITIVADA|2

In [ ]:
#para termos certeza que essa coluna não foi escrita no df:

df_3.show()

+------+------+---------+--------------------+-------------------+--------------------+----------+--------------+----------------+---------+------------------+----------+--------+---------+----------+--------+
|regiao|estado|municipio|             revenda|               cnpj|             nomerua|Numero Rua|   complemento|          bairro|      cep|           produto|datacoleta|vlrvenda|vlrcompra|unidmedida|bandeira|
+------+------+---------+--------------------+-------------------+--------------------+----------+--------------+----------------+---------+------------------+----------+--------+---------+----------+--------+
|     N|    AM|PARINTINS|F.J. COMERCIO DE ...| 09.368.602/0001-70|         RUA PARAIBA|        41|CONJ  MACURANY|  RAIMUNDO MUNIZ|69151-456|          GASOLINA|2024-08-01|    7.68|     NULL|R$ / litro|  BRANCA|
|     N|    AM|PARINTINS|F.J. COMERCIO DE ...| 09.368.602/0001-70|         RUA PARAIBA|        41|CONJ  MACURANY|  RAIMUNDO MUNIZ|69151-456|GASOLINA ADITIVADA|2

In [ ]:
#O que usamos de fatiamento no python/pandas, no Pyspark usamos a função substring;

#OBS: Diferente do python que quando trabalhamos com um array se inicia do '0', aqui no pyspark se inicia literalmente no '1';

#Nesse caso, vamos pegar do primeiro ao 3 caracter da string da coluna municipio e criar uma nova coluna chamada 'substring_function';

df_3.withColumn('substring_function', substring('municipio', 1, 3))

DataFrame[regiao: string, estado: string, municipio: string, revenda: string, cnpj: string, nomerua: string, Numero Rua: string, complemento: string, bairro: string, cep: string, produto: string, datacoleta: date, vlrvenda: float, vlrcompra: string, unidmedida: string, bandeira: string, substring_function: string]

In [ ]:
df_3.show()

+------+------+---------+--------------------+-------------------+--------------------+----------+--------------+----------------+---------+------------------+----------+--------+---------+----------+--------+
|regiao|estado|municipio|             revenda|               cnpj|             nomerua|Numero Rua|   complemento|          bairro|      cep|           produto|datacoleta|vlrvenda|vlrcompra|unidmedida|bandeira|
+------+------+---------+--------------------+-------------------+--------------------+----------+--------------+----------------+---------+------------------+----------+--------+---------+----------+--------+
|     N|    AM|PARINTINS|F.J. COMERCIO DE ...| 09.368.602/0001-70|         RUA PARAIBA|        41|CONJ  MACURANY|  RAIMUNDO MUNIZ|69151-456|          GASOLINA|2024-08-01|    7.68|     NULL|R$ / litro|  BRANCA|
|     N|    AM|PARINTINS|F.J. COMERCIO DE ...| 09.368.602/0001-70|         RUA PARAIBA|        41|CONJ  MACURANY|  RAIMUNDO MUNIZ|69151-456|GASOLINA ADITIVADA|2

In [ ]:
#Nesse caso, estamos pegando os 3 ultimos caracter da string

df_3.withColumn('substring_function', substring('municipio', -3, 3))

DataFrame[regiao: string, estado: string, municipio: string, revenda: string, cnpj: string, nomerua: string, Numero Rua: string, complemento: string, bairro: string, cep: string, produto: string, datacoleta: date, vlrvenda: float, vlrcompra: string, unidmedida: string, bandeira: string, substring_function: string]

In [ ]:
df_3.show()

+------+------+---------+--------------------+-------------------+--------------------+----------+--------------+----------------+---------+------------------+----------+--------+---------+----------+--------+
|regiao|estado|municipio|             revenda|               cnpj|             nomerua|Numero Rua|   complemento|          bairro|      cep|           produto|datacoleta|vlrvenda|vlrcompra|unidmedida|bandeira|
+------+------+---------+--------------------+-------------------+--------------------+----------+--------------+----------------+---------+------------------+----------+--------+---------+----------+--------+
|     N|    AM|PARINTINS|F.J. COMERCIO DE ...| 09.368.602/0001-70|         RUA PARAIBA|        41|CONJ  MACURANY|  RAIMUNDO MUNIZ|69151-456|          GASOLINA|2024-08-01|    7.68|     NULL|R$ / litro|  BRANCA|
|     N|    AM|PARINTINS|F.J. COMERCIO DE ...| 09.368.602/0001-70|         RUA PARAIBA|        41|CONJ  MACURANY|  RAIMUNDO MUNIZ|69151-456|GASOLINA ADITIVADA|2

In [ ]:
#Nesse caso, estamos selecionando a data e pegando o valor central, pois ai saberemos o mês e também pegando os 4 primeiros caracter, pois teremos o ano

#substring('coluna_queremos_contar', incio, quais ou qual queremos pegar (lenbrando que ele conta o numero que está)

#ex: array_numeros = [123456789] - substring('array_numeros, 3, 2 ) - ele começa contar do numero 3 e conta duas casa, incluindo o 3 -> resultado: 3 e 4



df_3 = df_3.withColumn('mes', substring('datacoleta', 6, 2)) \
           .withColumn('ano', substring('datacoleta', 1, 4))

In [ ]:
#Passando as duas colunas (mês e ano) para dados do tipo inteiro

df = df_3\
          .withColumn('mes', col('mes').cast(IntegerType()))\
          .withColumn('ano', col('ano').cast(IntegerType()))


In [ ]:
df.printSchema()

root
 |-- regiao: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- revenda: string (nullable = true)
 |-- cnpj: string (nullable = true)
 |-- nomerua: string (nullable = true)
 |-- Numero Rua: string (nullable = true)
 |-- complemento: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- cep: string (nullable = true)
 |-- produto: string (nullable = true)
 |-- datacoleta: date (nullable = true)
 |-- vlrvenda: float (nullable = true)
 |-- vlrcompra: string (nullable = true)
 |-- unidmedida: string (nullable = true)
 |-- bandeira: string (nullable = true)
 |-- mes: integer (nullable = true)
 |-- ano: integer (nullable = true)



In [ ]:
df.show(10)

+------+------+---------+--------------------+-------------------+--------------------+----------+--------------+--------------+---------+------------------+----------+--------+---------+----------+--------+---+----+
|regiao|estado|municipio|             revenda|               cnpj|             nomerua|Numero Rua|   complemento|        bairro|      cep|           produto|datacoleta|vlrvenda|vlrcompra|unidmedida|bandeira|mes| ano|
+------+------+---------+--------------------+-------------------+--------------------+----------+--------------+--------------+---------+------------------+----------+--------+---------+----------+--------+---+----+
|     N|    AM|PARINTINS|F.J. COMERCIO DE ...| 09.368.602/0001-70|         RUA PARAIBA|        41|CONJ  MACURANY|RAIMUNDO MUNIZ|69151-456|          GASOLINA|2024-08-01|    7.68|     NULL|R$ / litro|  BRANCA|  8|2024|
|     N|    AM|PARINTINS|F.J. COMERCIO DE ...| 09.368.602/0001-70|         RUA PARAIBA|        41|CONJ  MACURANY|RAIMUNDO MUNIZ|6915

In [ ]:
#Assim como no pandas, essa função criada serve no pyspark também
nome_funcao = udf(lambda x: x.split('-')[0])

outra forma de extrair dia, mês e ano de uma coluna que está formatada como date (to_date('coluna', format=YYYY-MM-DD)

In [ ]:
df.withColumn('dia', day('datacoleta'))\
  .withColumn('ano', year('datacoleta'))\
  .withColumn('mes', month('datacoleta'))\
  .show()

+------+------+---------+--------------------+-------------------+--------------------+----------+--------------+----------------+---------+------------------+----------+--------+---------+----------+--------+---+----+---+
|regiao|estado|municipio|             revenda|               cnpj|             nomerua|Numero Rua|   complemento|          bairro|      cep|           produto|datacoleta|vlrvenda|vlrcompra|unidmedida|bandeira|mes| ano|dia|
+------+------+---------+--------------------+-------------------+--------------------+----------+--------------+----------------+---------+------------------+----------+--------+---------+----------+--------+---+----+---+
|     N|    AM|PARINTINS|F.J. COMERCIO DE ...| 09.368.602/0001-70|         RUA PARAIBA|        41|CONJ  MACURANY|  RAIMUNDO MUNIZ|69151-456|          GASOLINA|2024-08-01|    7.68|     NULL|R$ / litro|  BRANCA|  8|2024|  1|
|     N|    AM|PARINTINS|F.J. COMERCIO DE ...| 09.368.602/0001-70|         RUA PARAIBA|        41|CONJ  MACU

Fazendo drop de colunas no pyspark

In [ ]:
df = df.drop(col('cnpj'),
        col('nomerua'),
        col('Numero Rua'),
        col('complemento'),
        col('bairro'),
        col('cep'),
        col('unidmedida')
)

In [ ]:
df.show()

+------+------+---------+--------------------+------------------+----------+--------+---------+--------+---+----+
|regiao|estado|municipio|             revenda|           produto|datacoleta|vlrvenda|vlrcompra|bandeira|mes| ano|
+------+------+---------+--------------------+------------------+----------+--------+---------+--------+---+----+
|     N|    AM|PARINTINS|F.J. COMERCIO DE ...|          GASOLINA|2024-08-01|    7.68|     NULL|  BRANCA|  8|2024|
|     N|    AM|PARINTINS|F.J. COMERCIO DE ...|GASOLINA ADITIVADA|2024-08-01|    7.68|     NULL|  BRANCA|  8|2024|
|    NE|    CE|ITAPIPOCA|SAO PEDRO COM VAR...|          GASOLINA|2024-08-01|    6.77|     NULL|     ALE|  8|2024|
|    NE|    CE|ITAPIPOCA|SAO PEDRO COM VAR...|GASOLINA ADITIVADA|2024-08-01|    6.78|     NULL|     ALE|  8|2024|
|    NE|    CE|ITAPIPOCA|SAO PEDRO COM VAR...|            ETANOL|2024-08-01|    5.24|     NULL|     ALE|  8|2024|
|    NE|    CE|ITAPIPOCA|COMERCIAL ITAPIPO...|          GASOLINA|2024-08-01|    6.79|   

In [ ]:
df.withColumn('mes', when )